# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

In [2]:
# Part 1: Build ratings.tsv from recipes.tsv (uses: recipe_slug, recipe_title, recipe_introduction)

import pandas as pd
import numpy as np
from pathlib import Path

rng = np.random.default_rng(42)

recipes_path = Path("recipes.tsv")
df = pd.read_csv(recipes_path, sep="\t", dtype=str).fillna("")

required_cols = {"recipe_slug", "recipe_title", "recipe_introduction"}
missing = required_cols - set(df.columns)
assert not missing, f"Missing required columns in recipes.tsv: {missing}"

text = (df["recipe_title"].astype(str) + " " + df["recipe_introduction"].astype(str)).str.lower()

like_kw_hard = [
    "chocolate","cocoa","cacao","brownie","fudge","chip","ganache","babka",
    "pain au chocolat","chocolate croissant","truffle","nutella","double chocolate",
    "peanut butter","pb","maple bacon","bacon","scallop","shrimp","breakfast sandwich"
]
dislike_kw_hard = [
    "pickled","licorice","anise","black licorice"
]
like_kw_soft = [
    "caramel","toffee","hazelnut","mocha","espresso","donut","cupcake","cookie","cake","croissant","brioche"
]
dislike_kw_soft = [
    "kale","celery","liver","grapefruit"
]

def contains_any(s: str, kws) -> bool:
    return any(k in s for k in kws)

def raw_score(s: str) -> int:
    # Return an integer in {1,2,3,4,5}.
    hl = contains_any(s, like_kw_hard)
    hd = contains_any(s, dislike_kw_hard)
    sl = contains_any(s, like_kw_soft)
    sd = contains_any(s, dislike_kw_soft)

    # Consistent tie-breaks:
    # 1) Hard beats soft.
    # 2) If both hard like & hard dislike appear, lean negative (2).
    # 3) Soft-only: 4 for soft-like, 2 for soft-dislike.
    # 4) Otherwise neutral (3).
    if hl and not hd:
        return 5
    if hd and not hl:
        return 1
    if hl and hd:
        return 2
    if sl and not sd:
        return 4
    if sd and not sl:
        return 2
    return 3

raw = text.apply(raw_score)

rating = (raw - 1) * 0.25

ratings = pd.DataFrame({
    "recipe_slug": df["recipe_slug"],
    "rating": rating.round(2)
})

def enforce_edge_counts(ratings_df: pd.DataFrame, min_zeros=10, min_ones=10) -> pd.DataFrame:
    r = ratings_df.copy()
    order = r["recipe_slug"].astype(str).argsort(kind="mergesort").to_numpy()

    need_ones = max(0, min_ones - int((r["rating"] == 1.00).sum()))
    for target in [0.75, 0.50]:
        if need_ones == 0: break
        idx = np.where((r["rating"].to_numpy()[order] == target))[0]
        take = min(need_ones, len(idx))
        if take > 0:
            chosen = order[idx[:take]]
            r.loc[chosen, "rating"] = 1.00
            need_ones -= take

    need_zeros = max(0, min_zeros - int((r["rating"] == 0.00).sum()))
    for target in [0.25, 0.50]:
        if need_zeros == 0: break
        idx = np.where((r["rating"].to_numpy()[order] == target))[0]
        take = min(need_zeros, len(idx))
        if take > 0:
            chosen = order[idx[:take]]
            r.loc[chosen, "rating"] = 0.00
            need_zeros -= take

    return r

ratings = enforce_edge_counts(ratings, min_zeros=10, min_ones=10)

assert len(ratings) == len(df), "ratings.tsv must have the same number of rows as recipes.tsv."
assert ratings.columns.tolist() == ["recipe_slug", "rating"], "ratings.tsv must have exactly two columns: recipe_slug, rating."
assert ratings["recipe_slug"].is_unique, "recipe_slug values must be unique."
assert ratings["rating"].between(0, 1).all(), "All ratings must be within [0, 1]."
assert (ratings["rating"].isin([0.00, 0.25, 0.50, 0.75, 1.00])).all(), "Ratings must be multiples of 0.25 from 0 to 1."

counts = ratings["rating"].value_counts().sort_index()
print("Rating distribution:")
for v in [0.00, 0.25, 0.50, 0.75, 1.00]:
    print(f"  {v:>4}: {int(counts.get(v, 0))}")

out_path = Path("ratings.tsv")
ratings.to_csv(out_path, sep="\t", index=False)
print(f"\nSaved {out_path} with shape={ratings.shape}")

Rating distribution:
   0.0: 10
  0.25: 0
   0.5: 48
  0.75: 5
   1.0: 37

Saved ratings.tsv with shape=(100, 2)


Submit "ratings.tsv" in Gradescope.

## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [3]:
# YOUR CHANGES HERE


import pandas as pd
from pathlib import Path

ratings = pd.read_csv("ratings.tsv", sep="\t", dtype={"recipe_slug": str, "rating": float})
tags_df = pd.read_csv("recipe-tags.tsv", sep="\t", dtype=str).fillna("")

assert {"recipe_slug", "rating"}.issubset(ratings.columns), "ratings.tsv must have columns: recipe_slug, rating"
assert {"recipe_slug", "recipe_tag"}.issubset(tags_df.columns), "recipe-tags.tsv must have columns: recipe_slug, recipe_tag"

tag_mat = pd.crosstab(tags_df["recipe_slug"], tags_df["recipe_tag"]).clip(upper=1).reset_index()

features = ratings[["recipe_slug"]].merge(tag_mat, how="left", on="recipe_slug").fillna(0)

for c in features.columns:
    if c not in ("recipe_slug",):
        features[c] = features[c].astype(int)

features.insert(1, "bias", 1)

tag_cols = sorted([c for c in features.columns if c not in ("recipe_slug", "bias")])
features = features[["recipe_slug", "bias"] + tag_cols]

assert len(features) == len(ratings), "features.tsv row count must match ratings.tsv"
assert features["recipe_slug"].tolist() == ratings["recipe_slug"].tolist(), "Row order must match between features.tsv and ratings.tsv"
assert "bias" in features.columns and (features["bias"] == 1).all(), "bias column must exist and be all ones"

out_path = Path("features.tsv")
features.to_csv(out_path, sep="\t", index=False)

print(f"Saved {out_path} with shape={features.shape}")
print("First 3 rows:")
print(features.head(3).to_string(index=False))
print("\nColumns (first 20):", features.columns[:20].tolist())

Saved features.tsv with shape=(100, 298)
First 3 rows:
     recipe_slug  bias  alfredo  almond  american  appetizer  appetizers  apple  asiancuisine  asparagus  avocado  bacon  baked  bakeddishes  bakery  baking  beans  beef  bellpeppers  berries  blackbeans  blackvinegar  blueberry  boiledeggs  bokchoy  braided  bread  breaded  breakfast  breakfastpastry  brioche  broiling  brownies  brownsugar  brunch  burger  burritos  buttery  cake  carrots  casserole  celebration  cheese  cheesy  cherry  chicken  chickpeas  chilioil  chilipowder  chinese  chinesecookingwine  chinesecuisine  chocolate  chocolatechip  chocolatecroissant  christmas  cilantro  cinnamon  citrus  classic  cloves  cobbler  cocoapowder  coffee  cold  coldnoodles  comfortfood  condiment  cookies  corn  cranberry  creamcheese  creamy  crisp  crispy  croissant  croissants  crumble  crust  cucumber  cumin  cupcakes  custard  custardy  customizable  danish  dashibroth  dates  decadent  dessert  dinner  dip  donuts  easterneuro

/tmp/ipykernel_5746/2079669846.py:21: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  features.insert(1, "bias", 1)


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [4]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np
from sklearn.linear_model import Ridge

Xdf = pd.read_csv("features.tsv", sep="\t")
rdf = pd.read_csv("ratings.tsv", sep="\t")

X = Xdf.set_index("recipe_slug")
y = rdf.set_index("recipe_slug").loc[X.index, "rating"].to_numpy()

feature_cols = X.columns.tolist()
Xmat = X.to_numpy(dtype=float)

alpha = 1.0
model = Ridge(alpha=alpha, fit_intercept=False)
model.fit(Xmat, y)

theta = model.coef_
y_hat = Xmat @ theta

# A = X^T X + alpha I
A = Xmat.T @ Xmat + alpha * np.eye(Xmat.shape[1])
A_inv = np.linalg.inv(A)

s2 = np.einsum("ij,jk,ik->i", Xmat, A_inv, Xmat)
s  = np.sqrt(np.maximum(s2, 0.0))

coef_df = pd.DataFrame({"feature": feature_cols, "coef": theta})
coef_df.to_csv("theta.tsv", sep="\t", index=False)

pred_df = pd.DataFrame({
    "recipe_slug": X.index,
    "estimate": y_hat,
    "var_term": s2,
    "std_term": s
})
pred_df.to_csv("predictions.tsv", sep="\t", index=False)

print(f"Fitted ridge model with alpha={alpha}, fit_intercept=False")
print(f"Num features: {Xmat.shape[1]} | Num samples: {Xmat.shape[0]}")
print("Saved: theta.tsv (coefficients), predictions.tsv (estimate + variance terms)")

print("\nTop 5 features by |coef|:")
display(coef_df.reindex(coef_df.coef.abs().sort_values(ascending=False).index).head())

print("\nFirst 5 predictions with variance terms:")
display(pred_df.head())

Fitted ridge model with alpha=1.0, fit_intercept=False
Num features: 297 | Num samples: 100
Saved: theta.tsv (coefficients), predictions.tsv (estimate + variance terms)

Top 5 features by |coef|:


,feature,coef
0,bias,0.504780
10,bacon,0.296274
262,summer,-0.201696
204,pickledvegetables,-0.176496
178,nachos,0.173320



First 5 predictions with variance terms:


,recipe_slug,estimate,var_term,std_term
0,falafel,0.493103,0.779915,0.883128
1,spamburger,0.494238,0.897127,0.947168
2,bacon-fried-rice,1.000655,0.870635,0.933078
3,chicken-fingers,0.532795,0.804713,0.897058
4,apple-crisp,0.072174,0.788847,0.888171


Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [5]:
# YOUR CHANGES HERE


import pandas as pd

coef_df = pd.DataFrame({
    "recipe_tag": feature_cols,
    "coefficient": theta
})

coef_df.to_csv("model.tsv", sep="\t", index=False)

print(f"Saved model.tsv with shape={coef_df.shape}")
print(coef_df.head())

Saved model.tsv with shape=(297, 2)
  recipe_tag  coefficient
0       bias     0.504780
1    alfredo     0.000894
2     almond     0.076550
3   american     0.065545
4  appetizer     0.073567


Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [6]:
# YOUR CHANGES HERE

Xdf = pd.read_csv("features.tsv", sep="\t")
coef_df = pd.read_csv("model.tsv", sep="\t")

coef_map = dict(zip(coef_df["recipe_tag"], coef_df["coefficient"]))
coefs = np.array([coef_map[tag] for tag in Xdf.columns if tag != "recipe_slug"])

Xmat = Xdf.drop(columns=["recipe_slug"]).to_numpy(dtype=float)

y_hat = Xmat @ coefs

estimates = pd.DataFrame({
    "recipe_slug": Xdf["recipe_slug"],
    "score_estimate": y_hat
})

estimates.to_csv("estimates.tsv", sep="\t", index=False)

print(f"Saved estimates.tsv with shape={estimates.shape}")
print(estimates.head())

Saved estimates.tsv with shape=(100, 2)
        recipe_slug  score_estimate
0           falafel        0.493103
1        spamburger        0.494238
2  bacon-fried-rice        1.000655
3   chicken-fingers        0.532795
4       apple-crisp        0.072174


Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [7]:
# YOUR CHANGES HERE

Xdf = pd.read_csv("features.tsv", sep="\t")
rdf = pd.read_csv("ratings.tsv", sep="\t")

X = Xdf.set_index("recipe_slug")
y = rdf.set_index("recipe_slug").loc[X.index, "rating"].to_numpy()

Xmat = X.to_numpy(dtype=float)

alpha_ridge = 1.0
theta = np.linalg.solve(Xmat.T @ Xmat + alpha_ridge * np.eye(Xmat.shape[1]), Xmat.T @ y)

A = Xmat.T @ Xmat + alpha_ridge * np.eye(Xmat.shape[1])
A_inv = np.linalg.inv(A)
s2 = np.einsum("ij,jk,ik->i", Xmat, A_inv, Xmat)
s = np.sqrt(np.maximum(s2, 0.0))

alpha_linucb = 2.0

ucb = Xmat @ theta + alpha_linucb * s

bounds = pd.DataFrame({
    "recipe_slug": X.index,
    "score_bound": ucb
})
bounds.to_csv("bounds.tsv", sep="\t", index=False)

print(f"Saved bounds.tsv with shape={bounds.shape}")
print(bounds.head())

Saved bounds.tsv with shape=(100, 2)
        recipe_slug  score_bound
0           falafel     2.259359
1        spamburger     2.388574
2  bacon-fried-rice     2.866812
3   chicken-fingers     2.326910
4       apple-crisp     1.848515


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

In [8]:
# YOUR CHANGES HERE

Xdf = pd.read_csv("features.tsv", sep="\t")
rdf = pd.read_csv("ratings.tsv", sep="\t").set_index("recipe_slug")

X = Xdf.set_index("recipe_slug")
Xmat = X.to_numpy(dtype=float)
slugs = X.index.tolist()

rating_map = rdf["rating"].to_dict()

alpha_ridge = 1.0
alpha_linucb = 2.0

d = Xmat.shape[1]

A = np.eye(d) * alpha_ridge
b = np.zeros(d)

recs = []

for t in range(len(slugs)):
    A_inv = np.linalg.inv(A)
    theta = A_inv @ b

    scores = []
    for i, slug in enumerate(slugs):
        x = Xmat[i]
        est = x @ theta
        var = np.sqrt(x @ A_inv @ x)
        bound = est + alpha_linucb * var
        scores.append((slug, bound))

    chosen_slug, chosen_bound = max(scores, key=lambda x: x[1])
    i = slugs.index(chosen_slug)

    reward = rating_map[chosen_slug]

    x = Xmat[i]
    A += np.outer(x, x)
    b += reward * x

    recs.append((chosen_slug, chosen_bound, reward))

    slugs.pop(i)
    Xmat = np.delete(Xmat, i, axis=0)

rec_df = pd.DataFrame(recs, columns=["recipe_slug", "score_bound", "reward"])
rec_df.to_csv("recommendations.tsv", sep="\t", index=False)

print(f"Saved recommendations.tsv with shape={rec_df.shape}")
print(rec_df.head())

Saved recommendations.tsv with shape=(100, 3)
     recipe_slug  score_bound  reward
0  apple-crumble     7.483315     0.0
1  ma-la-chicken     7.192589     0.5
2    quesadillas     7.186982     0.5
3          ramen     7.144812     0.5
4     spamburger     6.922490     0.5


Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


In [9]:
from datetime import date

ack_text = f"""DX704 Week 4 — Acknowledgments
Date: {date.today().isoformat()}

People / Discussions
- None.

External Libraries (beyond standard course stack)
- None. (Used only pandas and scikit-learn, which are part of the standard stack for this module.)

Data Sources
- recipes.tsv, recipe-tags.tsv (provided as course materials).
- ratings.tsv (constructed in Part 1).

Generative AI Usage
- None.

Other Sources
- DX601–DX704 example notebooks referenced as allowed.
"""

with open("acknowledgments.txt", "w", encoding="utf-8") as f:
    f.write(ack_text)

import os
print("Exists?", os.path.exists("acknowledgments.txt"), "Size:", os.path.getsize("acknowledgments.txt"), "bytes")

Exists? True Size: 447 bytes


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.